In [ ]:
from arcgis.gis import GIS
import platform
import psutil
import sys
import os
import shutil
import datetime
import time
from datetime import date, timedelta
import pandas as pd
import arcpy
from arcgis.features import GeoAccessor
import arcgis
from arcgis.raster import *
from arcgis.raster.analytics import *

arcgis.env.verbose=True

In [ ]:
sys.path.append(r'C:\GitHub\code-base')
# from agol_credentials import kv_oceans_username, kv_oceans_password, kv_oceans_org
from agol_credentials import esri_oceans_username, esri_oceans_password, esri_oceans_org
gis = GIS(esri_oceans_org, esri_oceans_username, esri_oceans_password, verify_cert=True)
gis

In [ ]:
print("**********Platform information**********")
print(f"System: {platform.system()}")
print(f"Node Name: {platform.node()}")
print(f"Release: {platform.release()}")
print(f"Version: {platform.version()}")
print(f"Machine: {platform.machine()}")
print(f"Processor: {platform.processor()}")
print("Memory information:")
mem = psutil.virtual_memory()
print(f"Total: {mem.total / (1024 ** 3):.2f} GB")
print(f"Available: {mem.available / (1024 ** 3):.2f} GB")
print(f"Used: {mem.used / (1024 ** 3):.2f} GB")
print(f"Percentage: {mem.percent}%")
print("CPU information:")
print(f"Physical cores: {psutil.cpu_count(logical=False)}")
print("**********Platform information**********")

In [ ]:
# gfwapi_path = '/arcgis/home/gfwapiclient'
# if not os.path.isdir(gfwapi_path):
#     %pip install --target /arcgis/home/gfwapiclient gfw-api-python-client
#     # %pip install gfw-api-python-client
#     # %pip install --upgrade gfw-api-python-client

In [ ]:
# sys.path.append(gfwapi_path)
import gfwapiclient as gfw

In [ ]:
# sys.path.append('/arcgis/home/notebook/gfw')
from access_credentials import access_token_key

In [ ]:
# workspace = "/arcgis/home/notebook/gfw"
workspace = r"C:\temp\gfw"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

if not os.path.exists(workspace):
    os.makedirs(workspace)

In [ ]:
access_token = os.environ.get(
    "GFW_API_ACCESS_TOKEN",
    access_token_key,
)

gfw_client = gfw.Client(
    access_token=access_token,
)

In [ ]:
# Explore GFW Client Fourwings capabilities
print("=== GFW Client Fourwings Methods ===")
fourwings_methods = [method for method in dir(gfw_client.fourwings) if not method.startswith('_')]
for method in fourwings_methods:
    print(f"- {method}")

print("\n=== Create Report Method Details ===")
# Get help for create_report method
help(gfw_client.fourwings.create_report)

print("\n=== Fourwings Object Type ===")
print(type(gfw_client.fourwings))

# You can also check the docstring
print("\n=== Create Report Docstring ===")
print(gfw_client.fourwings.create_report.__doc__)

# Inspect the method signature
import inspect
print("\n=== Create Report Method Signature ===")
signature = inspect.signature(gfw_client.fourwings.create_report)
print(f"Parameters: {signature}")
for param_name, param in signature.parameters.items():
    print(f"  - {param_name}: {param.annotation if param.annotation != inspect.Parameter.empty else 'No type hint'}")
    if param.default != inspect.Parameter.empty:
        print(f"    Default: {param.default}")

In [ ]:
global_geojson = {"type": "Polygon", "coordinates": [[[180, 90], [180, -90], [-180, -90], [-180, 90], [180, 90]]]}

In [ ]:
start = date(2020, 1, 1)
end = date.today()

date_list = []
d = start
while d <= end:
    date_list.append(d.strftime("%Y-%m-%d"))
    d += timedelta(days=1)

# print or use dates
for day in date_list:
    print(day)

In [ ]:
product_name = "sar_detection_daily_unmatched" ### keep it to three words (separated by underscores "_")
gfw_constant_raster = r"data/gfw_constant_raster.tif"

gdb_name = "gfw_" + product_name + ".gdb"
if not arcpy.Exists(gdb_name):
    arcpy.CreateFileGDB_management(workspace, gdb_name)

output_raster_workspace = os.path.join(workspace, product_name)
if not os.path.exists(output_raster_workspace):
    os.makedirs(output_raster_workspace)

In [ ]:
for current_date in date_list:
    print(f"Processing date: {current_date}")
    date_name = current_date.replace("-", "")
    next_date = (pd.to_datetime(current_date) + pd.Timedelta(days=1)).strftime("%Y-%m-%d")
    print(f"Next date for processing: {next_date}")
    feature_class = os.path.join(workspace, gdb_name, product_name + "_" + str(date_name))
    if arcpy.Exists(feature_class):
        print(f"Feature class {feature_class} already exists. Skipping.")
        continue
    try:
        print("Waiting 10 seconds to eliminate too many requests error...")
        time.sleep(10)
        dt_start = datetime.now()
        gfw_report = await gfw_client.fourwings.create_report(
            datasets=["public-global-sar-presence:latest"],
            spatial_resolution="HIGH",
            temporal_resolution="DAILY",
            group_by="GEARTYPE",
            matched='false',
            start_date=current_date,
            end_date=next_date,
            geojson=global_geojson,
        )
        dt_api = datetime.now()
        print(f"Elapsed time: {dt_api - dt_start} to query GFW API for Report.")
        gfw_report_df = gfw_report.df()
        gfw_report_df['lat'] = gfw_report_df['lat'] + 0.005
        gfw_report_df['lon'] = gfw_report_df['lon'] + 0.005
        dt_dataframe = datetime.now()
        print(f"Elapsed time: {dt_dataframe - dt_api} to create dataframe.")
        gfw_sedf = pd.DataFrame.spatial.from_xy(df = gfw_report_df, x_column = "lon", y_column = "lat", sr=4326)
        dt_sdef = datetime.now()
        print(f"Elapsed time: {dt_sdef - dt_api} to create spatially enabled dataframe.")
        gfw_sedf.spatial.to_featureclass(feature_class)
        dt_fc = datetime.now()
        print(f"Elapsed time: {dt_fc - dt_sdef} to create feature class.")
        print(f"Completed processing date {current_date} API to Feature Class.")

        output_raster = os.path.join(output_raster_workspace, product_name + '_' + str(date_name) + ".tif")
        
        arcpy.conversion.PointToRaster(
            in_features=feature_class,
            value_field="detections",
            out_rasterdataset=output_raster,
            cell_assignment="SUM",
            priority_field="NONE",
            cellsize=0.01,  # adjust cell size as needed 0.1 for LOW and 0.01 for HIGH
            build_rat="BUILD"
        )

        dt_ras = datetime.now()
        print(f"Elapsed time: {dt_ras - dt_fc} to create raster from points.")
        print(f"Completed processing date {current_date} Point to Raster.")

        
        final_raster = os.path.join(output_raster_workspace, product_name + '_' + str(date_name) + "_global.tif")
        with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
                              snapRaster=gfw_constant_raster, cellSize=gfw_constant_raster, extent=gfw_constant_raster):
            global_raster = arcpy.sa.Plus(
                in_raster_or_constant1=output_raster,
                in_raster_or_constant2=gfw_constant_raster
            )
            global_raster.save(final_raster)
        
        arcpy.management.Delete(output_raster)
        print(f"Deleted intermediate raster: {output_raster}")

        print(f"***** Completed processing date {current_date} *****")
    except Exception as e:
        print(f"Error processing date {current_date}: {e}")

        

In [ ]:
tif_files = [f for f in os.listdir(output_raster_workspace) if f.endswith('.tif')]
print(tif_files)

In [ ]:
md_path = f"{workspace}/{gdb_name}/{product_name}"
if not arcpy.Exists(md_path):
    arcpy.CreateMosaicDataset_management(
        in_workspace=f"{workspace}/{gdb_name}",
        in_mosaicdataset_name=product_name,
        coordinate_system=4326
    )
print(f"Mosaic dataset '{product_name}' created in {gdb_name}")

In [ ]:
arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset=f"{gdb_name}/{product_name}",
    raster_type="Raster Dataset",
    input_path=output_raster_workspace,
    update_cellsize_ranges="UPDATE_CELL_SIZES",
    update_boundary="UPDATE_BOUNDARY",
    update_overviews="NO_OVERVIEWS",
    maximum_pyramid_levels=None,
    maximum_cell_size=0,
    minimum_dimension=1500,
    spatial_reference=None,
    filter="*.TIF",
    sub_folder="NO_SUBFOLDERS",
    duplicate_items_action="EXCLUDE_DUPLICATES",
    build_pyramids="NO_PYRAMIDS",
    calculate_statistics="NO_STATISTICS",
    build_thumbnails="NO_THUMBNAILS",
    operation_description="",
    force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
    estimate_statistics="NO_STATISTICS",
    aux_inputs=None,
    enable_pixel_cache="NO_PIXEL_CACHE",
    cache_location=workspace
)

In [ ]:
arcpy.management.CalculateField(
    in_table=f"{gdb_name}/{product_name}",
    field="date_text",
    expression='!Name!.split("_")[4]',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
arcpy.management.CalculateField(
    in_table=f"{gdb_name}/{product_name}",
    field="sar_date",
    expression='str(!date_text!)[4:6] + "/" + str(!date_text!)[6:8] + "/" + str(!date_text!)[:4]',
    expression_type="PYTHON3",
    code_block="",
    field_type="DATE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
arcpy.management.CalculateField(
    in_table=f"{gdb_name}/{product_name}",
    field="variable_mapped",
    expression='"SAR Detection - Unmatched"',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
arcpy.md.BuildMultidimensionalInfo(
    in_mosaic_dataset=f"{gdb_name}/{product_name}",
    variable_field="variable_mapped",
    dimension_fields="sar_date date days",
    variable_desc_units="'SAR Detections' 'SAR Vessel Detections' count",
    delete_multidimensional_info="NO_DELETE_MULTIDIMENSIONAL_INFO"
)

In [ ]:
crf_path = os.path.join(workspace, product_name + ".crf")

with arcpy.EnvManager(parallelProcessingFactor="90%"):
    arcpy.management.CopyRaster(
        in_raster=f"{gdb_name}/{product_name}",
        out_rasterdataset=crf_path,
        config_keyword="",
        background_value=None,
        nodata_value="",
        onebit_to_eightbit="NONE",
        colormap_to_RGB="NONE",
        pixel_type="",
        scale_pixel_value="NONE",
        RGB_to_Colormap="NONE",
        format="CRF",
        transform="NONE",
        process_as_multidimensional="ALL_SLICES",
        build_multidimensional_transpose="TRANSPOSE"
    )

In [ ]:
with arcpy.EnvManager(parallelProcessingFactor="90%"):
    out_multidimensional_raster = arcpy.ia.AggregateMultidimensionalRaster(
        in_multidimensional_raster=crf_path,
        dimension="StdTime",
        aggregation_method="SUM",
        variables="'SAR Detection - Unmatched'",
        aggregation_def="INTERVAL_KEYWORD",
        interval_keyword="MONTHLY",
        interval_value=None,
        interval_unit="",
        interval_ranges=None,
        aggregation_function="",
        ignore_nodata="DATA",
        dimensionless="DIMENSIONS",
        percentile_value=90,
        percentile_interpolation_type="NEAREST"
    )
    out_multidimensional_raster.save(crf_path.replace('daily','monthly'))

In [ ]:
with arcpy.EnvManager(parallelProcessingFactor="90%"):
    out_multidimensional_raster = arcpy.ia.AggregateMultidimensionalRaster(
        in_multidimensional_raster=crf_path,
        dimension="StdTime",
        aggregation_method="SUM",
        variables="'SAR Detection - Unmatched'",
        aggregation_def="INTERVAL_KEYWORD",
        interval_keyword="YEARLY",
        interval_value=None,
        interval_unit="",
        interval_ranges=None,
        aggregation_function="",
        ignore_nodata="DATA",
        dimensionless="DIMENSIONS",
        percentile_value=90,
        percentile_interpolation_type="NEAREST"
    )
    out_multidimensional_raster.save(crf_path.replace('daily','annual'))

In [ ]:
def overwrite_til(til_item, ras):
    copy_raster_op_til = analytics.copy_raster(
        # input_raster=os.path.join(data_dir, ras),
        input_raster=ras,
        output_name=tiled_img_item,
        tiles_only=True,
        gis=gis
    )
    return copy_raster_op_til

In [ ]:
# search by title
items_by_title = gis.content.search(query=f'title:"{product_name}"', max_items=100)

if items_by_title:
    print("Found items by title:")
    for it in items_by_title:
        print(it.title, it.id, it.type, it.owner, it.url if hasattr(it, 'url') else "")
        tiled_img_item = gis.content.get(it.id)
        tiled_img_item
        in_ras = data_url
        result = None
        if arcpy.Exists(in_ras):
            print(f"Dataset exists: {in_ras} ...overwriting Imagery Layer")
            result = overwrite_til(tiled_img_item, in_ras)
        else:
            print(f"Dataset does not exist: {in_ras}")

if not items_by_title:
    print(f"No items found for '{product_name}' - creating Imagery Layer")
    data_url = crf_path
    print(data_url)

    ImageryLayer = copy_raster(input_raster=data_url,
                               output_name=product_name,
                               gis=gis)
    ImageryLayer